In [ ]:
from elasticsearch import Elasticsearch
client = Elasticsearch(["http://192.168.12.151:9200"])

In [ ]:
data = {
    "title": "My Document",
    "content": "This is the test2."
}

index_response = client.index(index="test_index", body=data)
print(f"Document indexed. Version: {index_response['_version']}")

# write into es

In [ ]:
for re in resault_v1:
    index_response = client.index(index="test_index_2", body=re)

# query es

In [ ]:
res = client.search(index="test_index_2", body={"query": {"match_all": {}}})

In [ ]:

# 处理查询结果
for hit in res['hits']['hits']:
    print(hit['_source'])

In [ ]:
# 查询所有数据
res = client.search(index="test_index_2", body={"query": {"match_all": {}}})

# 打印查询结果
for hit in res['hits']['hits']:
    print(hit['_source'])

# qurey index

In [ ]:
# all_indices = client.indices.get_alias().keys()
index_names = client.indices.get_alias().keys()

In [ ]:

for i in all_indices:
    print(i)

# Get data of txt

In [ ]:
import pandas as pd

def read_txt(pathfile):
    txt_file_path = pathfile
    df = pd.read_csv(txt_file_path, delimiter='\t')
    list_of_lists = df.values.tolist()
    return list_of_lists

In [ ]:
list_of_lists=read_txt('Data/guangdong_openstack.txt')

In [ ]:
list_of_lists

## cpu

In [ ]:
import datetime
def get_date_time_now():
    current_dt = datetime.datetime.now()
    formatted_dt = current_dt.strftime("%Y-%m-%d %H:%M:%S")
    # print("Formatted date and time: ", formatted_dt)
    return formatted_dt

In [ ]:
now=get_date_time_now()

In [ ]:
def get_cpu_data(file_path):
    now=get_date_time_now()
    list_of_lists_1=read_txt(file_path)
    cpulist=[]
    for i in list_of_lists_1:
        if '[cpu_load]' in i[0]:
            # print(i[0])
            for ii in i:
                cpudict={}
                iili=(ii.split())
                cpudict['nodeName']=iili[1]
                cpudict['cpuUsage']=round(float(iili[3])+float(iili[5]))
                cpudict['createTime']=now
                cpulist.append(cpudict)
    return cpulist

In [ ]:
def merge_dicts_with_same_nodename(cpulist, memlist):
    merged_dict = {} 

    
    for dict1 in cpulist:
        nodename = dict1.get('nodeName')  
        if nodename:
          
            if nodename not in merged_dict:
                merged_dict[nodename] = dict1
            
            else:
                merged_dict[nodename].update(dict1)

    
    for dict2 in memlist:
        nodename = dict2.get('nodeName')
        if nodename:
            if nodename not in merged_dict:
                merged_dict[nodename] = dict2
            else:
                merged_dict[nodename].update(dict2)

    return list(merged_dict.values())  


## disk

In [ ]:
def get_disk_data(file_path):
    now=get_date_time_now()
    list_of_lists_1=read_txt(file_path)
    disklist=[]
    for i in list_of_lists_1:
        # print(i)
        if '[disk]' in i[0]:
            # print(i)
            for ii in i:
                diskdict={}
                iili=(ii.split())
                # print(iili)
                diskdict={
                    'nodeName':iili[1],
                    # '':iili[3],
                    'sysDiskUsage':iili[9].split("%")[0],
                    'createTime':now}
                disklist.append(diskdict)
    return disklist

In [ ]:
def get_disk_ansy(file_path):
    data=get_disk_data(file_path)
    merged_data = {}
    for item in data:
        node_name = item['nodeName']
        if node_name in merged_data:
            merged_data[node_name]['sysDiskUsage'].append(int(item['sysDiskUsage']))
        else:
            merged_data[node_name] = item.copy()
            merged_data[node_name]['sysDiskUsage'] = [int(item['sysDiskUsage'])]  # 创建新的列表存储值

    for item in merged_data.values():
        item['sysDiskUsage'] = list(set(item['sysDiskUsage']))

    merged_list = list(merged_data.values())
    return merged_list
# print(final_result)

In [ ]:
newdata=get_disk_ansy("Data/guangdong_node.txt")

In [ ]:
newdata

## mem

In [ ]:
def get_mem_data(file_path):
    now=get_date_time_now()
    list_of_lists_1=read_txt(file_path)
    memlist=[]
    for i in list_of_lists_1:
        # print(i)
        if '[mem]' in i[0]:
            memdict={}
            # print(i)
            for ii in i:
                iili=(ii.split())
                # print(iili)
                mem=int(iili[5].split("G")[0])/int(iili[3].split("G")[0])
                memdict['nodeName']=iili[1]
                memdict['memUsage']=round(mem*100)
                memdict['createTime']=now
                memlist.append(memdict)
    return memlist


## down or up

In [ ]:
def get_host_down_up(file_path,province_name,pool_name):
    now=get_date_time_now()
    list_of_lists_1=read_txt(file_path)
    downlist=[]
    for i in list_of_lists_1:
        if 'reacheable' in i[0] and 'ping' not in i[0]:
            for ii in i:
                downdict={}
                iili=(ii.split())
                downdict['nodeName']=iili[0]
                print(iili[1])
                if iili[1]=='reacheable.':
                    downdict['hostIcmpDown'] = True
                else:
                    downdict['hostIcmpDown'] = False
                # downdict['hostIcmpDown']=iili[1]
                downdict['province']=province_name
                downdict['pool']=pool_name
                downdict['sysDiskInodeUsage']=None
                downdict['otherDiskUsage']=None
                downdict['otherDiskInodeUsage']=None
                downdict['loadAverage5']=None
                downdict['createTime']=now
                downlist.append(downdict)
    return downlist

# mergent

In [ ]:
cpulist=get_cpu_data("Data/guangdong_node.txt")
memlist=get_mem_data("Data/guangdong_node.txt")
downlist=get_host_down_up("Data/guangdong_node.txt",'广东','广东')
final_result=get_disk_ansy("Data/guangdong_node.txt")

In [ ]:
merged_result = merge_dicts_with_same_nodename(cpulist, memlist)
merged_result_v1 = merge_dicts_with_same_nodename(merged_result,downlist)
resault_v1=merge_dicts_with_same_nodename(merged_result_v1,final_result)

In [ ]:
resault_v1

In [ ]:
for re in resault_v1:
    index_response = client.index(index="os_system_monitor", body=re)

# openstack service

In [ ]:
import pandas as pd

txt_file_path = 'Data/guangdong_openstack.txt'
df = pd.read_csv(txt_file_path, delimiter='\t')
list_of_lists = df.values.tolist()
list_of_lists=read_txt("Data/guangdong_openstack.txt")

In [ ]:
def get_nova_status(file_path,province_name,pool_name):
    now=get_date_time_now()
    list_of_lists=read_txt(file_path)
    novalist=[]
    for lis in list_of_lists:
        # print(lis)
        for info in lis:
            if 'nova' in info:
                # print(info.split())
                novadict={}
                listll=info.split()
                hostname=[s for s in listll if "aqc" in s]
                # print(hostname)
                if (info.split()[-1]) == 'up':
                    novadict['province']=province_name
                    novadict['pool']=pool_name
                    novadict['glance']=None
                    novadict['swift']=None
                    novadict['keystone']=None
                    novadict['horizon']=None
                    novadict['heat']=None
                    novadict['ceilometer']=None
                    novadict['manila']=None
                    novadict['rabbitmqActive']=None
                    novadict['rabbitmqLeader']=None
                    novadict['nova']=True
                    novadict['nodeName']=hostname[0]
                    novadict['createTime']=now
                    novalist.append(novadict)
                else:
                    novadict['province']=province_name
                    novadict['pool']=pool_name
                    novadict['glance']=None
                    novadict['swift']=None
                    novadict['keystone']=None
                    novadict['horizon']=None
                    novadict['heat']=None
                    novadict['ceilometer']=None
                    novadict['manila']=None
                    novadict['rabbitmqActive']=None
                    novadict['rabbitmqLeader']=None
                    novadict['nova']=False
                    novadict['nodeName']=hostname[0]
                    novadict['createTime']=now
                    novalist.append(novadict)
    return novalist

## netron

In [ ]:

def get_data_methods(keyName,judgeinfo,file_path):
    now=get_date_time_now()
    list_of_lists=read_txt(file_path)
    neutronlist=[]
    for lis in list_of_lists:
        # print(lis)
        for info in lis:
            if keyName in info:
                # print(info.split())
                neutrondict={}
                listll=info.split()
                hostname=[s for s in listll if "aqc" in s]
                # print(listll)
                if listll[-1]==judgeinfo:
                    neutrondict[keyName]=True
                    neutrondict['nodeName']=hostname[0]
                    neutrondict['createTime']=now
                    neutronlist.append(neutrondict)
                else:
                    neutrondict[keyName]=False
                    neutrondict['nodeName']=hostname[0]
                    neutronlist.append(neutrondict)
    neutronlist_v1=[dict(t) for t in {tuple(sorted(d.items())) for d in neutronlist}]
    return neutronlist_v1


## cinder

In [ ]:
cinder_list=get_data_methods("openstack cinder","up","Data/guangdong_openstack.txt")
cinder_list = [{k.replace('openstack ', ''): v if k != 'openstack cinder' else v for k, v in d.items()} for d in cinder_list]

## mergend

## merget methods

In [ ]:
def merget_list_dict_methods(list1,list2):
    merged_list = list1 + list2
    merged_dict = {}
    for d in merged_list:
        key = d.get('nodeName') or d.get('nameNode')  
        if key in merged_dict:
            merged_dict[key].update(d)
        else:
            merged_dict[key] = d
    result = list(merged_dict.values())
    return result

In [ ]:
cinder_list=get_data_methods("openstack cinder","up","Data/guangdong_openstack.txt")
cinder_list = [{k.replace('openstack ', ''): v if k != 'openstack cinder' else v for k, v in d.items()} for d in cinder_list]
neutronlist_v=get_data_methods("neutron",'":-)"',"Data/guangdong_openstack.txt")
novalist=get_nova_status("Data/guangdong_openstack.txt",'广东','广东')
unique_dicts_nova_list = [dict(t) for t in {tuple(sorted(d.items())) for d in novalist}]
result=merget_list_dict_methods(neutronlist_v,unique_dicts_nova_list)
result_vvv=merget_list_dict_methods(result,cinder_list)

In [112]:
result_vvv
for dictionary in result_vvv:
    # 检查是否存在 "province" 键
    if "province" not in dictionary:
        # 如果不存在，则添加键值对
        dictionary["province"] = "广东"
    elif "pool" not in dictionary:
        dictionary["pool"]="广东"

In [113]:
result_vvv

[{'createTime': '2024-03-15 17:11:23',
  'neutron': True,
  'nodeName': 'aqc-gd01-crh-kvm-10e5e76e19',
  'ceilometer': None,
  'glance': None,
  'heat': None,
  'horizon': None,
  'keystone': None,
  'manila': None,
  'nova': True,
  'pool': '广东',
  'province': '广东',
  'rabbitmqActive': None,
  'rabbitmqLeader': None,
  'swift': None},
 {'createTime': '2024-03-15 17:11:23',
  'neutron': True,
  'nodeName': 'aqc-gd01-crh-kvm-10e5e76e11',
  'ceilometer': None,
  'glance': None,
  'heat': None,
  'horizon': None,
  'keystone': None,
  'manila': None,
  'nova': True,
  'pool': '广东',
  'province': '广东',
  'rabbitmqActive': None,
  'rabbitmqLeader': None,
  'swift': None},
 {'createTime': '2024-03-15 17:11:23',
  'neutron': True,
  'nodeName': 'aqc-gd01-crh-kvm-10e5e76e22',
  'ceilometer': None,
  'glance': None,
  'heat': None,
  'horizon': None,
  'keystone': None,
  'manila': None,
  'nova': True,
  'pool': '广东',
  'province': '广东',
  'rabbitmqActive': None,
  'rabbitmqLeader': None,
  's

In [115]:
for re in result_vvv:
    index_response = client.index(index="open_stack_monitor", body=re)

## rabbit mq

In [ ]:
## has mergent to other 

## write into es

In [ ]:
for re in result_vvv:
    index_response = client.index(index="open_stack_monitor", body=re)

In [ ]:
res = client.search(index="open_stack_monitor", body={"query": {"match_all": {}}})

# network

In [ ]:
def get_network_status(file_path,pronvice_name,pool_name):
    now=get_date_time_now()
    txtinfo=read_txt(file_path)
    network_list=[]
    for i in txtinfo:
        # print(i)
        for ii in i:
            # print(ii)
            ip_reach={}
            if 'reacheable.' in ii:
                iii=(ii.split())
                # print(iii)
                if iii[-1]=='reacheable.':
                    ip_reach['businessNetPing']=True
                    # network_list.append(ip_reach)
                    ip_reach['storageNetPing']=None
                    ip_reach['province']=pronvice_name
                    ip_reach['pool']=pool_name
                    ip_reach['nodeName']=iii[0]
                    ip_reach['createTime']=now
                    network_list.append(ip_reach)
                else:
                    ip_reach['businessNetPing']=False
                    # network_list.append(ip_reach)
                    ip_reach['storageNetPing']=None
                    ip_reach['province']=pronvice_name
                    ip_reach['pool']=pool_name
                    ip_reach['nodeName']=iii[0]
                    ip_reach['createTime']=now
                    network_list.append(ip_reach)
    return network_list
        

In [ ]:
network_list=get_network_status("Data/guangdong_node.txt",'广东','广东')

In [ ]:
network_list

### write into es

In [ ]:
for re in network_list:
    index_response = client.index(index="network_monitor", body=re)

In [ ]:
res = client.search(index="network_monitor", body={"query": {"match_all": {}}})

In [ ]:
res

# ceph

In [ ]:
def get_ceph_status(file_path,province_name,pool_name):
    now=get_date_time_now()
    ceph_info=read_txt(file_path)
    ceph_dict = {}
    pool=[]

    for i in ceph_info:
        for ii in i:
         
            if '[ceph-pool]' in ii:
                pool_dict={}
            
                iii=ii.split()
        
                # pool_dict[iii[2]]=float(iii[4])
                pool.append(round(float(iii[4])))
    ceph_dict['cephOsdDfUsage']=pool
    ceph_dict['province']=province_name
    ceph_dict['pool']=pool_name
    ceph_dict['cephSlowRequest']=False
    ceph_dict['cephCopiesNum']=None
    ceph_dict['cephDataLowerRank']=False
    ceph_dict['createTime']=now
    return ceph_dict

In [ ]:
ceph_dict=get_ceph_status("Data/guangdong_ceph.txt",'广东','广东')

In [ ]:
ceph_dict

In [ ]:

index_response = client.index(index="ceph_monitor", body=ceph_dict)

In [ ]:
import ssl
print(ssl.OPENSSL_VERSION)

In [ ]:
# now = get_date_time_now()
list_of_lists_1 = read_txt("Data/guangdong_node.txt")
disklist = []

for i in list_of_lists_1:
    # print(i)
    if '[disk]' in i[0]:
        # print(i)
        for ii in i:
            diskdict = {}
            disk_v=[]
            iili = (ii.split())
            print(iili)
            disk_v.append(int(iili[9].split("%")[0]))